In [1]:
from linear import *
from convolutional import *
from activations import *
from loss import CategoricalCrossEntropyLoss
import mnist

np.random.seed(0)

In [2]:
class MnistConv:
    def __init__(self, lr = 0.01) -> None:
        self.layers = [
            ConvLayer(1, 32, 5, activaton=ReLu()),
            MaxPool(2),
            Flatten(),
            Linear(32*12**2, 64, Sigmoid()),
            Linear(64, 10, None)
        ]
        self.lr = lr
        self.res = None
    
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        self.res = x
        return x
    
    def backward(self, expected_output):
        loss = CategoricalCrossEntropyLoss()
        loss_val = loss(self.res, expected_output)
        err = loss.derivatives()
        for layer in reversed(self.layers):
            err = layer.backward(err, self.lr)
        return loss_val

In [3]:
x_train, t_train, x_test, t_test = mnist.load()
model = MnistConv()

In [4]:
EPOCHES = 3

for epoch in range(EPOCHES):
    print(f"epoch {epoch}/{EPOCHES}")
    loss_val = 0
    for i, (img, label) in enumerate(zip(x_train, t_train)):

        res = model.forward(img.reshape((1, 1, 28, 28))/255.0)
        loss_val += model.backward(label)
        if i % 6000 == 0 and i > 0:
            print(f"epoch: {epoch}.{i//6000}, loss: {loss_val/6000}")
            loss_val = 0




epoch 0/3
epoch: 0.1, loss: 1.2800874304934964
epoch: 0.2, loss: 0.7344519608392068
epoch: 0.3, loss: 0.658066691984265
epoch: 0.4, loss: 0.547573084073393
epoch: 0.5, loss: 0.575709893340678
epoch: 0.6, loss: 0.5663411528953972
epoch: 0.7, loss: 0.4736537406703636
epoch: 0.8, loss: 0.5023362560985972
epoch: 0.9, loss: 0.5103001539766407
epoch 1/3
epoch: 1.1, loss: 0.41613088492660555
epoch: 1.2, loss: 0.40069498509373513
epoch: 1.3, loss: 0.43465365758475943
epoch: 1.4, loss: 0.3479516539678411
epoch: 1.5, loss: 0.40850393425053894
epoch: 1.6, loss: 0.4061290864667609
epoch: 1.7, loss: 0.3715212503752108
epoch: 1.8, loss: 0.39686237637771776
epoch: 1.9, loss: 0.37271340876648906
epoch 2/3
epoch: 2.1, loss: 0.34594245337745755
epoch: 2.2, loss: 0.3514578356331633
epoch: 2.3, loss: 0.35791449817347326
epoch: 2.4, loss: 0.29751558132017986
epoch: 2.5, loss: 0.3199872592068827
epoch: 2.6, loss: 0.3016144889189218
epoch: 2.7, loss: 0.2935946364061692
epoch: 2.8, loss: 0.3236257106341769
ep

In [5]:
results = np.zeros((3, 10))
for (img, label) in zip(x_test, t_test):
    results[0][label] += 1
    r = model.forward(img.reshape((1, 1, 28, 28))/255.0)
    if np.argmax(r, axis = 1) == label:
        results[1][label] +=1
    results[2][np.argmax(r, axis = 1)] += 1

print(results[1])
print(results[0])
print(results[2])

[ 955. 1110.  910.  949.  831.  787.  892.  909.  842.  950.]
[ 980. 1135. 1032. 1010.  982.  892.  958. 1028.  974. 1009.]
[1025. 1140.  975. 1100.  885.  849.  960.  966.  904. 1196.]


In [6]:
np.sum(results[1])/np.sum(results[0])

0.9135

In [7]:
results[1]/results[0]

array([0.9744898 , 0.97797357, 0.88178295, 0.93960396, 0.84623218,
       0.882287  , 0.93110647, 0.88424125, 0.86447639, 0.94152626])